In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm, colors
from time import time
import warnings
import numpy as np
import numpy.linalg as la
warnings.filterwarnings('ignore')
%matplotlib qt

In [2]:
def golden_section(x1, Si, x2, eps):
    a = x2[0]
    b = x2[1]
    tau = 0.381967
    alpha1 = a*(1-tau) + b*tau
    alpha2 = a*tau + b*(1-tau)
    falpha1 = f(x1+alpha1*Si)
    falpha2 = f(x1+alpha2*Si)
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1
            alpha1 = alpha2
            falpha1 = falpha2
            alpha2 = tau*a + (1-tau)*b
            falpha2 = f(x1+alpha2*Si)
        else:
            b = alpha2
            alpha2 = alpha1
            falpha2 = falpha1
            alpha1 = tau*b + (1-tau)*a
            falpha1 = f(x1+alpha1*Si)
        if np.abs(f(x1+alpha1*Si)- f(x1+alpha2*Si)) < eps :
            break;
    return alpha1, falpha1

In [3]:
def f(x):
    #fx = x[0]- x[1] + 2*x[0]**2 + 2*x[0]*x[1] + x[1]**2
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    #fx = 100*(x[1]-x[0]**2)**2+(1-x[0])**2
    return fx
    #fx = 100*(x[1]-x[0]**2)**2+(1-x[0])**2

def gradiente(x, dx):
    grad = np.zeros(2)
    grad[0] = (f([x[0]+dx,x[1]])- f([x[0]-dx,x[1]]))/(2*dx)
    grad[1] = (f([x[0],x[1]+dx])- f([x[0],x[1]-dx]))/(2*dx)
    return grad

In [4]:
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2]) #segunda función
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False)
plt.show()



In [5]:
fx = lambda xx1, xx2, Fx1, Fx2, k1, k2: k1*(((np.sqrt((np.power(xx1,2))+(np.power((xx2+1),2))))-1)**2) + k2*(((np.sqrt((np.power(xx1,2))+(np.power((xx2+1),2))))-1)**2) - ((Fx1*xx1)+(Fx2*xx2))
derivada1_1 = lambda xx1, xx2, Fx1, Fx2, k1, k2, dx: (f(xx1+dx, xx2, Fx1, Fx2, k1, k2)- f(xx1-dx, xx2, Fx1, Fx2, k1, k2))/(2*dx)
derivada1_2 = lambda xx1, xx2, Fx1, Fx2, k1, k2, dx: (f(xx1, xx2+dx, Fx1, Fx2, k1, k2)- f(xx1, xx2-dx, Fx1, Fx2, k1, k2))/(2*dx)
x1 = np.arange(-1,1,0.001)
x2 = np.arange(-1,1,0.001)
xx1, xx2 = np.meshgrid(x1, x2) 
dx = 0.01
dy = 0.01 
fx1 = 20
fx2 = 40
k1 = 100
k2 = 90

In [6]:
#Inicialización de parametros
#Delta para las derivadas
dx = 0.01
#Epsilon de parada
e = 0.001

In [7]:
#Test the point Xi+1 for optimality. If ||∇fi+1|| ≤ 𝜀, where 𝜀 is a small quantity,
#take X* ≈ X i+1 and stop the process. Otherwise, go to step 5.
#1. Start with an initial point X1
xi = [-1,1]
xi_1 = xi
#n × n positive definite symmetric matrix [B1] as an initial estimate of the inverse of the Hessian matrix of f
#In the absence of additional information, [B1] is taken as the identity matrix [I].
Bi = np.eye(len(xi))
print('-------------------------------------')
print("Valor inicial de la funcion:")
print('-------------------------------------')
print(f(xi))
fx_prev=f(xi)
print('-------------------------------------')
HistorialPuntos = []
print(str('i')+'\t'+str('x1')+'\t\t\t'+str('x2')+'\t\t\t'+str('fx')+'\t\t\t'+str('normal'))
for i in range(1,1000):
    #Compute the gradient vector ∇f1 = ∇f(X1) and set the iteration number as i = 1.
    gF_xi = gradiente(xi, dx)
    #2. Compute the gradient of the function, ∇fi, at point Xi, and set
    Si = np.matmul(np.dot(1, Bi),gF_xi)
    #3. Find the optimal step length λ∗i in the direction Si and set
    alpha, fx = golden_section(xi_1,Si,xi,e)
    xi_1 = xi + np.dot(alpha, Si)
    #4. Test the point Xi+1 for optimality. If ||∇fi+1|| ≤ 𝜀, where 𝜀 is a small quantity, take X* ≈ X i+1 and stop the process. Otherwise, go to step 5.
    if(la.norm(gF_xi)<e or abs(fx-fx_prev)<e):
        print(str(i)+'\t'+str(xi[0])+'\t'+str(xi[1])+'\t'+str(fx)+'\t'+str(la.norm(gF_xi)))
        #print('adios putitos')
        break
    #Update the Hessian matrix as
    else:
        g1 = np.reshape(gradiente(xi_1, dx),(2,1)) - np.reshape(gF_xi,(2,1))
        d1 = np.reshape(np.array(xi),(2,1)) - np.reshape(xi_1,(2,1))
        d1d1t = np.dot(d1, np.reshape(d1,(1,2)))
        d1tg1 = np.matmul(d1.transpose(),g1)
        d1g1t = np.matmul(d1, g1.transpose())
        g1d1t = np.matmul(g1, d1.transpose())
        g1tb1g1 = np.matmul(np.matmul(g1.transpose(), Bi), g1)
        d1g1tb1 = np.matmul(d1g1t,Bi)
        b1g1d1t = np.matmul(Bi, g1d1t)
        Bi = Bi + ((1+(g1tb1g1/d1tg1))*(d1d1t/d1tg1)) - (d1g1tb1/d1tg1) - (b1g1d1t/d1tg1) 
    HistorialPuntos.append([[xi_1[0],xi[0]],[xi_1[1],xi[1]]])

    xi = xi_1
    fx_prev = fx
    print(str(i)+'\t'+str(xi[0])+'\t'+str(xi[1])+'\t'+str(fx)+'\t'+str(la.norm(gF_xi)))
    
x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
xx, yy = np.meshgrid(x, y)
z = f([xx,yy])
plt.contour(x,y,z,20)
plt.plot(-1,-1, 'ro--',  markersize=6)
plt.plot(0.5,0, 'go--', markersize=6)

for pts in HistorialPuntos:
    plt.plot(pts[0],pts[1], 'r', linewidth=2)
    #print(pts)


-------------------------------------
Valor inicial de la funcion:
-------------------------------------
270.29416855008
-------------------------------------
i	x1			x2			fx			normal
1	-0.38818553753387286	-0.01087414992637381	8.942560728437048	444.3170606463973
2	0.02789282660098752	0.19963280241336254	-0.946434106790278	33.7986428109748
3	0.0619551923671787	0.19307412402226315	-1.7608872306864223	40.321299318039806
4	0.1093179921915165	0.16863019477589666	-3.196829993534555	37.5374368796027
5	0.1599538349540478	0.1564864886959654	-4.128121938639824	29.22197827261738
6	0.19629678169184883	0.14069314381776743	-4.84287732043295	25.659735929158398
7	0.24392447765708591	0.1257971415796549	-5.525268167763016	21.37345904019856
8	0.2902471373111293	0.10484967063120805	-6.149507274126659	18.167517768531496
9	0.3423070371312805	0.08552363529675741	-6.637390069859254	13.811261754452957
10	0.3852618676632617	0.061725196927104	-6.989708829839637	10.931176308155571
11	0.4281655172984588	0.04418767

In [8]:
#Test the point Xi+1 for optimality. If ||∇fi+1|| ≤ 𝜀, where 𝜀 is a small quantity,
#take X* ≈ X i+1 and stop the process. Otherwise, go to step 5.
#1. Start with an initial point X1
xi = [-1,1]
xi_1 = xi
#n × n positive definite symmetric matrix [B1] as an initial estimate of the inverse of the Hessian matrix of f
#In the absence of additional information, [B1] is taken as the identity matrix [I].
Bi = np.eye(len(xi))
print('-------------------------------------')
print("Valor inicial de la funcion:")
print('-------------------------------------')
print(f(xi))
fx_prev=f(xi)
print('-------------------------------------')
print(str('i')+'\t'+str('x1')+'\t\t\t'+str('x2')+'\t\t\t'+str('fx')+'\t\t\t'+str('normal'))
for i in range(1,1000):
    Ba = Bi
    #Compute the gradient vector ∇f1 = ∇f(X1) and set the iteration number as i = 1.
    gF_xi = gradiente(xi, dx)
    #2. Compute the gradient of the function, ∇fi, at point Xi, and set
    Si = np.matmul(np.dot(-1, Bi),gF_xi)
    #print(Si)
    #3. Find the optimal step length λ∗i in the direction Si and set
    alpha, fx = golden_section(xi_1,Si,xi,e)
    xi_1 = xi + np.dot(alpha, Si)
    #print(fx)
    #4. Test the point Xi+1 for optimality. If ||∇fi+1|| ≤ 𝜀, where 𝜀 is a small quantity, take X* ≈ X i+1 and stop the process. Otherwise, go to step 5.
    if(la.norm(gF_xi)<e or abs(fx-fx_prev)<e):
        print(str(i)+'\t'+str(xi[0])+'\t'+str(xi[1])+'\t'+str(fx)+'\t'+str(la.norm(fx)))
        #print('adios putitos')
        break
    #Update the Hessian matrix as
    else:
        g1 = np.reshape(gradiente(xi_1, dx),(2,1)) - np.reshape(gF_xi,(2,1))
        d1 = np.reshape(np.array(xi_1),(2,1)) - np.reshape(xi,(2,1))
        d1d1t = np.matmul(d1, np.reshape(d1,(1,2)))
        d1tg1 = np.matmul(d1.transpose(),g1)
        d1g1t = np.matmul(d1, g1.transpose())
        g1d1t = np.matmul(g1, d1.transpose())
        g1tb1g1 = np.matmul(np.matmul(g1.transpose(), Bi), g1)
        d1g1tb1 = np.matmul(d1g1t,Bi)
        b1g1d1t = np.matmul(Bi, g1d1t)
        Bi = Ba + ((1+(g1tb1g1/d1tg1))*(d1d1t/d1tg1)) - (d1g1tb1/d1tg1) - (b1g1d1t/d1tg1)  
    #Ba = Bi
    #print(xi)
    #print(xi_1)
    xi = xi_1
    fx_prev = fx
    print(str(i)+'\t'+str(xi[0])+'\t'+str(xi[1])+'\t'+str(f([xi[0],xi[1]]))+'\t'+str(la.norm(gF_xi)))

-------------------------------------
Valor inicial de la funcion:
-------------------------------------
270.29416855008
-------------------------------------
i	x1			x2			fx			normal
1	-0.3891964002260401	-0.009203945922830359	8.942422925996079	444.3170606463973
2	-0.6667965095541379	-0.1497020971234556	20.55729874598661	33.680834472644165
3	-0.8785268755260057	-0.2679858411163769	32.20397587551467	42.02397724629367
4	-1.1217370784677485	-0.44554337096152385	52.25394433246648	62.11392099666701
5	-1.420384559404702	-0.6875134809621917	95.13104363972379	105.62997411497328
6	-1.870871995626276	-1.0350876415811394	223.0292688351073	188.64405630545951
7	-2.8184146761732434	-1.645489900881693	801.883363082706	354.02667265467255
8	-5.764941102020689	-3.17456137947082	5303.95652664901	747.9496317886045
9	-20.936411917200623	-10.12191540886436	91428.53750700322	1994.4062847475818
10	-223.61961947156294	-100.5930715167209	11301321.756007973	8332.539299580862
11	-22630.236706139618	-10091.7084730